In [1]:
import numpy as np
import theano
from theano import tensor as T
theano_rng = T.shared_randomstreams.RandomStreams(1234)

Using gpu device 0: Tesla K40c (CNMeM is disabled, cuDNN 5105)
/home/ekhongl/.conda/envs/py3/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
W_values = np.array([[1,1],[1,1]], dtype=theano.config.floatX)
bvis_values = np.array([0,0], dtype=theano.config.floatX)
bhid_values = np.array([0,0], dtype=theano.config.floatX)
W = theano.shared(W_values)
vbias = theano.shared(bvis_values)
hbias = theano.shared(bhid_values)

In [3]:
def propup(vis, v0_doc_len):
        pre_sigmoid_activation = T.dot(vis, W) + T.dot(hbias,v0_doc_len).T        #---------------------------[edited]
        return [pre_sigmoid_activation, T.nnet.sigmoid(pre_sigmoid_activation)]

def sample_h_given_v(v0_sample, v0_doc_len):
    pre_sigmoid_h1, h1_mean = propup(v0_sample, v0_doc_len)
    h1_sample = theano_rng.binomial(size=h1_mean.shape,
                                         n=1, p=h1_mean,
                                         dtype=theano.config.floatX)
    return [pre_sigmoid_h1, h1_mean, h1_sample]

def propdown(hid):
    pre_softmax_activation = T.dot(hid, W.T) + vbias                     #---------------------------[edited]
    return [pre_softmax_activation, T.nnet.softmax(pre_softmax_activation)]

def sample_v_given_h(h0_sample, v0_doc_len):
    pre_softmax_v1, v1_mean = propdown(h0_sample)
    v1_sample = theano_rng.multinomial(size=None,
                                         n=v0_doc_len, pvals=v1_mean,
                                         dtype=theano.config.floatX)     #---------------------------[edited]
    v1_doc_len = v1_sample.sum(axis=1).reshape([1,v1_sample.shape[0]])
    return [pre_softmax_v1, v1_mean, v1_sample, v1_doc_len]

def gibbs_hvh(h0_sample, v0_doc_len):
    pre_softmax_v1, v1_mean, v1_sample, v1_doc_len = sample_v_given_h(h0_sample, v0_doc_len)
    pre_sigmoid_h1, h1_mean, h1_sample = sample_h_given_v(v1_sample, v0_doc_len)
    return [pre_sigmoid_h1, h1_mean, h1_sample,
            pre_softmax_v1, v1_mean, v1_sample, v1_doc_len]                          #---------------------------[edited]

def gibbs_vhv(v0_sample, v0_doc_len):
    pre_sigmoid_h1, h1_mean, h1_sample = sample_h_given_v(v0_sample, v0_doc_len)
    pre_softmax_v1, v1_mean, v1_sample, v1_doc_len  = sample_v_given_h(h1_sample, v0_doc_len)
    return [pre_sigmoid_h1, h1_mean, h1_sample,
            softmax_v1, v1_mean, v1_sample, v1_doc_len]                              #---------------------------[edited]

In [4]:
ipt = T.matrix()
ipt_rSum = ipt.sum(axis=1).reshape([1,ipt.shape[0]])

In [5]:
pre_sigmoid_ph, ph_mean, ph_sample = sample_h_given_v(ipt, ipt_rSum)
chain_start = ph_sample

In [6]:
([pre_sigmoid_nhs,
  nh_means,
  nh_samples,
  pre_sigmoid_nvs,
  nv_means,
  nv_samples,
  nv_samples_sum],   updates) = theano.scan(
                                gibbs_hvh,
                                outputs_info=[None, None, chain_start, None, None, None, ipt_rSum],
                                n_steps=1,
                                name="gibbs_hvh" )

In [7]:
gibbs = theano.function( [ipt], outputs=[pre_sigmoid_nvs,
                                          nv_means,
                                          nv_samples,
                                          nv_samples_sum,
                                          pre_sigmoid_nhs,
                                          nh_means,
                                          nh_samples], updates=updates )

In [10]:
b = np.array([[2,0,],[0,2],[1,1]], dtype = theano.config.floatX)
gibbs(b)

ValueError: dimension mismatch in args to gemm (1,2)x(2,2)->(2,1)
Apply node that caused the error: GpuGemm{no_inplace}(<CudaNdarrayType(float32, matrix)>, TensorConstant{1.0}, GpuFromHost.0, GpuDimShuffle{1,0}.0, TensorConstant{1.0})
Toposort index: 26
Inputs types: [CudaNdarrayType(float32, matrix), TensorType(float32, scalar), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), TensorType(float32, scalar)]
Inputs shapes: [(2, 1), (), (1, 2), (2, 2), ()]
Inputs strides: [(1, 0), (), (0, 1), (1, 2), ()]
Inputs values: [b'CudaNdarray([[ 0.]\n [ 0.]])', array(1.0, dtype=float32), b'CudaNdarray([[ 1.  1.]])', b'CudaNdarray([[ 1.  1.]\n [ 1.  1.]])', array(1.0, dtype=float32)]
Outputs clients: [[GpuDimShuffle{0,1,x,x}(GpuGemm{no_inplace}.0), GpuDimShuffle{x,0,1}(GpuGemm{no_inplace}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'

In [22]:
b = np.array([[2,0,],[0,2],[1,1]], dtype = theano.config.floatX)
[pre_sigmoid_h1, h1_mean, h1_sample, softmax_v1, v1_doc_len, v1_mean, v1_sample]= gibbs(b)
print(pre_sigmoid_h1)
print('-------')
print(h1_mean)
print('-------')
print(h1_sample)
print('-------')
print(softmax_v1)
print('-------')
print(v1_mean)
print('-------')
print(v1_sample)

ValueError: dimension mismatch in args to gemm (3,2)x(2,2)->(2,1)
Apply node that caused the error: GpuGemm{no_inplace}(<CudaNdarrayType(float32, matrix)>, TensorConstant{1.0}, GpuFromHost.0, GpuDimShuffle{1,0}.0, TensorConstant{1.0})
Toposort index: 8
Inputs types: [CudaNdarrayType(float32, matrix), TensorType(float32, scalar), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), TensorType(float32, scalar)]
Inputs shapes: [(2, 1), (), (3, 2), (2, 2), ()]
Inputs strides: [(1, 0), (), (2, 1), (1, 2), ()]
Inputs values: [b'CudaNdarray([[ 0.]\n [ 0.]])', array(1.0, dtype=float32), 'not shown', b'CudaNdarray([[ 1.  1.]\n [ 1.  1.]])', array(1.0, dtype=float32)]
Outputs clients: [[GpuDimShuffle{0,1,x,x}(GpuGemm{no_inplace}.0), GpuDimShuffle{x,0,1}(GpuGemm{no_inplace}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does no

In [193]:
v1_sample.shape
h1_mean[None,:,:].shape
h1_mean.shape[1:]
h1_mean.reshape(h1_mean.shape[1:]).shape

(3, 2)

In [11]:
W_values.sum(axis=1).reshape([1,W_values.sum(axis=1).shape[0]]).T

array([[ 2.],
       [ 2.]], dtype=float32)